In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/comptage/comptage-velo-donnees-compteurs.csv', sep=';')
df = df[['Identifiant du site de comptage', 'Date et heure de comptage', 'Comptage horaire']]

liste_identifiants = []
with open('data/comptage/identifiants_balises.txt', 'r') as file:
    for line in file:
        liste_identifiants.append(int(line[:-1]))

df = df[df['Identifiant du site de comptage'].isin(liste_identifiants)].reset_index(drop=True)

In [3]:
df.columns = ['id', 'date', 'comptage']

In [16]:
new_data = {}

df = df[~df['comptage'].isna()]

from tqdm import tqdm
for ind in tqdm(df.index):

    date = df['date'][ind]
    new_data[date] = new_data.get(date, 0) + df['comptage'][ind]

new_df = {
    'datetime': list(new_data.keys()),
    'compte': list(new_data.values())
}

new_df = pd.DataFrame(new_df)
new_df['year'] = new_df['datetime'].apply(lambda x: x[:4])
new_df = new_df[new_df['year'] == '2022'].reset_index(drop=True)

new_df['datetime'] = pd.to_datetime(new_df['datetime'], utc=True)
new_df['date'] = [str(d.date()) for d in new_df['datetime']]
new_df['time'] = [str(d.time()) for d in new_df['datetime']]

new_df.head()


100%|██████████| 831907/831907 [00:13<00:00, 63710.13it/s]


,datetime,compte,year,date,time
0,2021-12-31 23:00:00+00:00,1357.0,2022,2021-12-31,23:00:00
1,2022-01-01 06:00:00+00:00,482.0,2022,2022-01-01,06:00:00
2,2022-01-01 19:00:00+00:00,2608.0,2022,2022-01-01,19:00:00
3,2022-01-01 23:00:00+00:00,839.0,2022,2022-01-01,23:00:00
4,2022-01-02 00:00:00+00:00,409.0,2022,2022-01-02,00:00:00


In [17]:
new_df = new_df.iloc[1:].sort_values(by=['date', 'time']).reset_index(drop=True)

In [19]:
new_df = new_df[['date', 'time', 'compte']]

In [13]:
df_meteo = pd.read_csv('data/meteo/meteo_2022.csv',sep=';')
df_meteo = df_meteo[df_meteo['region (name)'] == 'Île-de-France']
df_meteo = df_meteo[['Date', 'Température', 'Humidité']].reset_index(drop=True)
df_meteo.columns = ['date', 'temperature', 'humidite']

In [14]:
import datetime as dt 

df_meteo['datetime'] = pd.to_datetime(df_meteo['date'], utc=True)
df_meteo['date'] = [str(d.date()) for d in df_meteo['datetime']]
df_meteo['time'] = [str(d.time()) for d in df_meteo['datetime']]

df_meteo = df_meteo.sort_values(by=['date', 'time']).reset_index(drop=True)[['date', 'time', 'temperature', 'humidite']]

df_meteo.head()

,date,time,temperature,humidite
0,2022-01-01,00:00:00,282.85,99.0
1,2022-01-01,03:00:00,282.05,100.0
2,2022-01-01,06:00:00,280.95,100.0
3,2022-01-01,09:00:00,280.65,100.0
4,2022-01-01,12:00:00,285.35,81.0


In [20]:
df_merged = pd.merge(left=new_df, right=df_meteo, how='left', left_on=['date', 'time'], right_on=['date', 'time'])

df_merged.head()

,date,time,compte,temperature,humidite
0,2022-01-01,00:00:00,1626.0,282.85,99.0
1,2022-01-01,01:00:00,1962.0,NaN,NaN
2,2022-01-01,02:00:00,1628.0,NaN,NaN
3,2022-01-01,03:00:00,1289.0,282.05,100.0
4,2022-01-01,04:00:00,825.0,NaN,NaN


In [21]:
columns = ['temperature', 'humidite']

df_index = df_merged[~df_merged['temperature'].isna()]
list_index = list(df_index.index)

for k in tqdm(range(len(list_index)-1)):
    index_start, index_end = list_index[k], list_index[k+1]
    ecart = index_end-index_start
    temp_0, temp_1 = df_index['temperature'][index_start], df_index['temperature'][index_end]
    humi_0, humi_1 = df_index['humidite'][index_start], df_index['humidite'][index_end]
    for j in range(1, ecart):
        df_merged['temperature'][index_start+j] = temp_0 + (temp_1-temp_0)*(j/ecart)
        df_merged['humidite'][index_start+j] = humi_0 + (humi_1-humi_0)*(j/ecart)

  0%|          | 0/2918 [00:00<?, ?it/s]C:\Users\yanbri\AppData\Local\Temp\ipykernel_18652\488691284.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['temperature'][index_start+j] = temp_0 + (temp_1-temp_0)*(j/ecart)
C:\Users\yanbri\AppData\Local\Temp\ipykernel_18652\488691284.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['humidite'][index_start+j] = humi_0 + (humi_1-humi_0)*(j/ecart)
100%|██████████| 2918/2918 [02:05<00:00, 23.18it/s]


In [23]:
df_merged.to_csv('2022_processed.csv', sep=';', index=None)

In [24]:
def annee_bissextile(year: int) -> bool:

    return ((year % 4 == 0 and year % 100 != 0) or (year % 400 == 0))


days_per_month = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
    7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}

def convert_date(date: str) -> float:

    is_bissextile = annee_bissextile(int(date[:4]))
    month = int(date[5:7])
    day = int(date[8::])

    nb_days = day if month <= 2 else day + is_bissextile
    total = 365 + is_bissextile

    for k in range(1, month):
        nb_days += days_per_month[k]
    
    return nb_days/total

def convert_hour(time: str) -> float:

    hour = int(time[:2])
    minutes = int(time[3:4])/15

    return (4*hour + minutes)/96

In [25]:
from math import pi, cos, sin

df_merged['angle_time'] = df_merged['time'].apply(lambda x: 2*pi*convert_hour(x))
df_merged['angle_date'] = df_merged['date'].apply(lambda x: 2*pi*convert_date(x))

df_merged['cos_date'] = df_merged['angle_date'].apply(cos)
df_merged['sin_date'] = df_merged['angle_date'].apply(sin)

df_merged['cos_time'] = df_merged['angle_time'].apply(cos)
df_merged['sin_time'] = df_merged['angle_time'].apply(sin)

df_merged = df_merged.drop(columns=['angle_time', 'angle_date'])

df_merged.head()

,date,time,compte,temperature,humidite,cos_date,sin_date,cos_time,sin_time
0,2022-01-01,00:00:00,1626.0,282.850000,99.000000,0.999852,0.017213,1.000000,0.000000
1,2022-01-01,01:00:00,1962.0,282.583333,99.333333,0.999852,0.017213,0.965926,0.258819
2,2022-01-01,02:00:00,1628.0,282.316667,99.666667,0.999852,0.017213,0.866025,0.500000
3,2022-01-01,03:00:00,1289.0,282.050000,100.000000,0.999852,0.017213,0.707107,0.707107
4,2022-01-01,04:00:00,825.0,281.683333,100.000000,0.999852,0.017213,0.500000,0.866025


In [13]:
df_merged.to_csv('data.csv', sep=';', index=None)

In [26]:
import datetime as dt

def is_week_end(date_str: str) -> int:

    weekday = dt.date.fromisoformat(date_str).isoweekday()
    return int(weekday == 6 or weekday == 7)

df_merged['week_end'] = df_merged['date'].apply(lambda x: is_week_end(x))

In [27]:
df_vacation = pd.read_csv('data/data_vacances.csv', sep=',')[['date', 'vacances_zone_c']]

liste_days = list(df_vacation[df_vacation['vacances_zone_c'] == True].date.unique())

df_merged['vacation'] = df_merged['date'].apply(lambda x: int(x in liste_days))

In [28]:
df_feries = pd.read_csv('data/jours_feries_metropole.csv', sep=',')
liste_feries = df_feries['date'].unique()

df_merged['ferie'] = df_merged['date'].apply(lambda x: int(x in liste_feries))

In [29]:
df_merged.head()

,date,time,compte,temperature,humidite,cos_date,sin_date,cos_time,sin_time,week_end,vacation,ferie
0,2022-01-01,00:00:00,1626.0,282.850000,99.000000,0.999852,0.017213,1.000000,0.000000,1,1,1
1,2022-01-01,01:00:00,1962.0,282.583333,99.333333,0.999852,0.017213,0.965926,0.258819,1,1,1
2,2022-01-01,02:00:00,1628.0,282.316667,99.666667,0.999852,0.017213,0.866025,0.500000,1,1,1
3,2022-01-01,03:00:00,1289.0,282.050000,100.000000,0.999852,0.017213,0.707107,0.707107,1,1,1
4,2022-01-01,04:00:00,825.0,281.683333,100.000000,0.999852,0.017213,0.500000,0.866025,1,1,1


In [40]:
df_merged.to_csv('2022_processed.csv', sep=';', index=None)

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [2]:
def eval(y_predict, y_test):
    mae = mean_absolute_error(y_predict, y_test)
    mse = mean_squared_error(y_predict, y_test)

    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {mse ** 0.5:.2f}")

In [3]:
df = pd.read_csv('data_processed.csv', sep=';')

data = df.dropna()
data = data.set_index(["date", "time"])

In [4]:
data["mean_24h"] = data["compte"].shift(4).rolling(24*4).mean()
data["max_24h"] = data["compte"].shift(4).rolling(24*4).max()
data["min_24h"] = data["compte"].shift(4).rolling(24*4).min()
data["mean_30d"] = data["compte"].shift(4).rolling(30*24*4).mean()
data["max_30d"] = data["compte"].shift(4).rolling(30*24*4).max()
data["min_30d"] = data["compte"].shift(4).rolling(30*24*4).min()

data = pd.concat([data.loc[:'2020-03-17'], data.loc['2020-06-11':]])
data_train = data.loc[:'2021-09-30'].dropna()
data_test = data.loc['2021-10-01':].dropna()

In [5]:
y_train = data_train[['compte']]
y_test = data_test[['compte']]

X_train = data_train[data_train.columns.difference(pd.Index(["compte"]))]
X_test = data_test[data_test.columns.difference(pd.Index(["compte"]))]

In [5]:
SEED = 42

y = data[['compte']]
X = data[data.columns.difference(pd.Index(["compte"]))]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = SEED)

In [6]:
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)

y_predict = regressor.predict(X_test)

eval(y_predict, y_test)

C:\Users\yanbri\AppData\Local\Temp\ipykernel_16720\4042571414.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train, y_train)


MAE: 523.66
RMSE: 861.21


In [7]:
X.shape

(94055, 15)